- Computes Mel-frequencies and deltas
- Computes MFCC and deltas
- Creates train and test sets for both Mel and MFCC features

In [2]:
import librosa
from matplotlib import pyplot as plt
import librosa.display
import IPython.display as ipd
import os
import audioread
from matplotlib.pyplot import specgram
import numpy as np
import pandas as pd
import re
import random as rndm
from random import shuffle

#path for audio files folder:
relevant_path = './data/cats_dogs/'

# get files from directory and do all or a few, depending on range extracted below
sound_file_paths = [relevant_path+f for f in os.listdir(relevant_path)]

files_id=[]
pattern='(\w*_\d*).wav'
for n in sound_file_paths:
    files_id.append(re.search(pattern, n ).group(1))


#print(files_id)
n_cats=164 # number of files with cat sounds
n_dogs=113 # number of files with barking sounds    
catLabel=0
dogLabel=1

In [3]:

def load_data( files_path, sr=None): 
        
    cats = []
    cats_sr=[]
    channel_cats = []
    dogs = []
    channel_dogs = []
    dogs_sr=[]
   
    for i in range(n_cats):
       # print(files_path[i])
        cat_i, sr_i = librosa.load(files_path[i],sr=sr)
        cats.append(cat_i)
        cats_sr.append(sr_i)
        with audioread.audio_open(files_path[i]) as input_file:
            channel_cats.append(input_file.channels)
   
    for j in range(n_cats, n_cats + n_dogs):
       # print(files_path[j])
        dog_j, sr_j = librosa.load(files_path[j],sr=sr)
        dogs.append(dog_j)
        dogs_sr.append(sr_j)
        with audioread.audio_open(files_path[j]) as input_file:
            channel_dogs.append(input_file.channels)

    #channel_cats = list(channel_cats)
    #channel_dogs = list(channel_dogs)
    
    return cats, cats_sr, channel_cats, dogs, dogs_sr, channel_dogs


#Feature extraction and dataframe manipulation functions:


# extract mel_spectograms:
# Compute_mel_frequencies function:
# Computes the Mel-scaled spectrograms for each audio file
# input: lists of raw audio for cats and dogs, and sampling rate
#output: melspectograms for each file, in a list per class
def compute_mel_frequencies(cats,dogs, sr ):
    cats_mel_frequencies = []
    dogs_mel_frequencies = []
    
    for c in cats:
        cats_mel_frequencies.append(librosa.feature.melspectrogram(y=c,sr=sr))
    for d in dogs:
        dogs_mel_frequencies.append(librosa.feature.melspectrogram(y=d,sr=sr))
        
    return cats_mel_frequencies,dogs_mel_frequencies

def compute_MEL_deltas(cats_mel_frequencies,dogs_mel_frequencies):
    cats_mel_deltas = []
    dogs_dogs_deltas = []
    
    for i in range(164):
        cats_mel_deltas.append(librosa.feature.delta(cats_mel_frequencies[i]))
    for i in range(113):
        dogs_dogs_deltas.append(librosa.feature.delta(dogs_mel_frequencies[i]))
   
    return cats_mel_deltas, dogs_dogs_deltas

# extract MFCCs:
def compute_mfccs(cats_mel_frequencies,dogs_mel_frequencies,sr=22050):
    cats_mfccs = []
    dogs_mfccs = []
    
    for i in range(n_cats):
        cats_mfccs.append(librosa.feature.mfcc(S=librosa.power_to_db(cats_mel_frequencies[i]),sr=sr))
    for i in range(n_dogs):
        dogs_mfccs.append(librosa.feature.mfcc(S=librosa.power_to_db(dogs_mel_frequencies[i]),sr=sr))
        
    return cats_mfccs,dogs_mfccs

def compute_MFCC_deltas(cats_mfccs,dogs_mfccs):
    cats_deltas = []
    dogs_deltas = []
    
    for i in range(164):
        cats_deltas.append(librosa.feature.delta(cats_mfccs[i]))
    for i in range(113):
        dogs_deltas.append(librosa.feature.delta(dogs_mfccs[i]))
        
    return cats_deltas,dogs_deltas



#to access a specific cell > dataframe.iloc[row]['column?name']


# GENERIC DATAFRAME that accepts any size features:
#features_name='mel', 'mel_delta', 'mfccs', 'mfcc_delta'
#features_cats is a list with the features extracted for each file.
def build_features_Dataframe(features_name, features_cats, features_dogs, files_id, removeSilentFrames=False, labelSilentFrames=False):
        
    df_cats=pd.DataFrame()
    for f in range(0, len(features_cats)):
        df_filec=pd.DataFrame([[np.transpose(features_cats[f])[0]]])
        for frame_mels in range(1,np.transpose(features_cats[f]).shape[0]):
            df_filec=df_filec.append([[np.transpose(features_cats[f])[frame_mels]]], ignore_index= True)
   
        df_filec['file_id']=files_id[f]
        df_cats=df_cats.append(df_filec)    

    df_cats['label']=catLabel
    df_cats.columns = [features_name, 'File_id', 'Label']

    df_dogs=pd.DataFrame()
    for fd in range(0, len(features_dogs)):
        df_file=pd.DataFrame([[np.transpose(features_dogs[fd])[0]]])
        for frame_mels in range(1,np.transpose(features_dogs[fd]).shape[0]):
            df_file=df_file.append([[np.transpose(features_dogs[fd])[frame_mels]]], ignore_index= True)

        df_file['file_id']=files_id[fd+n_cats]
        df_dogs=df_dogs.append(df_file)    

    df_dogs['label']=dogLabel
    df_dogs.columns = [features_name, 'File_id', 'Label']
    return df_cats, df_dogs




def split_dataframes(cats_dataframe,dogs_dataframe,test_size=0.3):

    #keep unbalanced dataset in the training for now: 0.7*164 cats+ 0.7*113 dogs
    n_train_files_cats=(int((1-test_size)*n_cats))
    n_train_files_dogs=(int((1-test_size)*n_dogs))
    
    # select 30% as test and 70% as train at file level! 
    files_id_cats=files_id[0:n_cats]
    files_id_dogs=files_id[n_cats:n_cats+n_dogs]
    rndm.shuffle(files_id_cats)

    files_id_cats_train=[]
    files_id_cats_test=[]
    for i in range(0,n_train_files_cats):
        files_id_cats_train.append(files_id_cats[i])
    files_id_cats_test=files_id_cats[n_train_files_cats:]

    
    rndm.shuffle(files_id_dogs)    
    files_id_dogs_train=[]
    files_id_dogs_test=[]
    for i in range(0,n_train_files_dogs):
        files_id_dogs_train.append(files_id_dogs[i])
    files_id_dogs_test=files_id_dogs[n_train_files_dogs:]

    dftrain_cats=cats_dataframe.loc[cats_dataframe['File_id'].isin(files_id_cats_train)]
    dftest_cats=cats_dataframe.loc[cats_dataframe['File_id'].isin(files_id_cats_test)]
    dftrain_dogs=dogs_dataframe.loc[dogs_dataframe['File_id'].isin(files_id_dogs_train)]
    dftest_dogs=dogs_dataframe.loc[dogs_dataframe['File_id'].isin(files_id_dogs_test)]

    # concatenate in two dataframe test and train.
    df_TRAIN=pd.DataFrame()
    df_TEST=pd.DataFrame()
    df_TRAIN=df_TRAIN.append(dftrain_cats)
    df_TRAIN=df_TRAIN.append(dftrain_dogs)
    df_TEST=df_TEST.append(dftest_cats)
    df_TEST=df_TEST.append(dftest_dogs)


    # # shuffle...
    df_TEST=df_TEST.sample(frac=1)
    df_TRAIN=df_TRAIN.sample(frac=1)
    return df_TRAIN, df_TEST

In [4]:
#load the files:
Cats_audio, cats_sr, channel_cats, Dogs_audio, dogs_sr, channel_dogs= load_data( sound_file_paths, sr=None)
sr=16000
#Feature computation

cats_mel_frequencies,dogs_mel_frequencies=compute_mel_frequencies(Cats_audio,Dogs_audio , sr)
cats_mel_deltas, dogs_mel_deltas=compute_MEL_deltas(cats_mel_frequencies,dogs_mel_frequencies)
cats_mfccs, dogs_mfccs= compute_mfccs(cats_mel_frequencies,dogs_mel_frequencies,sr)
cats_mfcc_delta, dogs_mfcc_delta=compute_MFCC_deltas(cats_mfccs,dogs_mfccs)

In [5]:
#Features Dataframes per class:

df_cats_melDeltas, df_dogs_melDeltas =build_features_Dataframe('Mel_deltas',cats_mel_deltas, dogs_mel_deltas, files_id )
df_cats_mel, df_dogs_mel =build_features_Dataframe('Mel',cats_mel_frequencies, dogs_mel_frequencies, files_id )
df_cats_MFCCDeltas, df_dogs_MFCCDeltas =build_features_Dataframe('MFCC_deltas',cats_mfcc_delta, dogs_mfcc_delta, files_id )
df_cats_MFCC, df_dogs_MFCC =build_features_Dataframe('MFCC',cats_mfccs, dogs_mfccs, files_id )


# Features concatenation dataframes:
df_cats_melANDdeltas=pd.DataFrame()
df_cats_MFCC_AND_Deltas=pd.DataFrame()
df_dogs_melANDdeltas=pd.DataFrame()
df_dogs_MFCC_AND_Deltas=pd.DataFrame()

df_cats_melANDdeltas=pd.concat([df_cats_mel, df_cats_melDeltas['Mel_deltas']], axis=1 )
df_cats_MFCC_AND_Deltas = pd.concat([df_cats_MFCC, df_cats_MFCCDeltas['MFCC_deltas']] , axis=1 )
df_dogs_melANDdeltas = pd.concat([df_dogs_mel, df_dogs_melDeltas['Mel_deltas']] , axis=1)
df_dogs_MFCC_AND_Deltas = pd.concat([df_dogs_MFCC, df_dogs_MFCCDeltas['MFCC_deltas']],axis=1 )

In [6]:
#creates Test and Train Sets and saves the dataframes:

df_TRAIN_final=pd.DataFrame() 
df_TEST_final=pd.DataFrame() 
df_TRAIN_final, df_TEST_final = split_dataframes( df_cats_melANDdeltas, df_dogs_melANDdeltas, test_size=0.3)
df_TRAIN_final.to_pickle('.\Features_sets/'+ 'Train_MELandDeltas.pkl')  
df_TEST_final.to_pickle('.\Features_sets/'+ 'Test_MELandDeltas.pkl')


df_TRAIN_final=pd.DataFrame() 
df_TEST_final=pd.DataFrame() 
df_TRAIN_final, df_TEST_final = split_dataframes( df_cats_MFCC_AND_Deltas, df_dogs_MFCC_AND_Deltas, test_size=0.3)
df_TRAIN_final.to_pickle('.\Features_sets/'+ 'Train_MFCCandDeltas.pkl')  
df_TEST_final.to_pickle('.\Features_sets/'+ 'Test_MFCCandDeltas.pkl')